In [1]:
import cv2
import numpy as np
import glob

### 1

In [2]:
image = cv2.imread('images/img1.png')

In [ ]:
cv2.namedWindow("Image1", cv2.WINDOW_NORMAL)
cv2.imshow('Image1',image,)
cv2.waitKey()

با برسی خطوط لبه میز میتوان متوجه اعوجاج تصویر شد و دلیل آن نیز نوع لنز میباشد.

### 2

In [4]:
ret, corners = cv2.findChessboardCorners(image, (24,17), None)

### 3

In [5]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [6]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [7]:
corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)

In [ ]:
cv2.namedWindow("Corners", cv2.WINDOW_NORMAL)
cv2.drawChessboardCorners(image, (24,17), corners2, ret)
cv2.imshow('Corners', image)
cv2.waitKey()

### 4

In [2]:
objp = np.zeros((17*24,3), np.float32)
print(objp)
objp[:,:2] = np.mgrid[0:24,0:17].T.reshape(-1,2)
print(objp)
objpoints = []
objpoints.append(objp)
imgpoints = []
imgpoints.append(corners)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]
 ...
 [0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[ 0.  0.  0.]
 [ 1.  0.  0.]
 [ 2.  0.  0.]
 ...
 [21. 16.  0.]
 [22. 16.  0.]
 [23. 16.  0.]]


NameError: name 'corners' is not defined

In [10]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

### 5

In [11]:
print('ret: ' + str(ret))
print('mtx: ' + str(mtx))
print('dist: ' + str(dist))
print('rvecs:' + str(rvecs))
print('tvecs: ' + str(tvecs))

ret: 0.4085045614486528
mtx: [[1.01020017e+03 0.00000000e+00 7.82625473e+02]
 [0.00000000e+00 1.02003650e+03 5.07602310e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist: [[-0.20773456  0.19897834  0.00076827 -0.01504764 -0.17925782]]
rvecs:(array([[-0.02464362],
       [ 0.0372673 ],
       [-0.00955078]]),)
tvecs: (array([[-23.81186856],
       [ 17.32668341],
       [ 60.67367447]]),)


### 6

In [12]:
img = cv2.imread('images/img5.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
x, y, w, h = roi
dst1 = dst[y:y+h, x:x+w]
cv2.namedWindow("Calibrated", cv2.WINDOW_NORMAL)
cv2.imshow('Calibrated', dst1)
cv2.waitKey()

### 7

In [14]:
objp = np.zeros((17*24,3), np.float32)
objp[:,:2] = np.mgrid[0:24,0:17].T.reshape(-1,2)

objpoints = []
imgpoints = []
images = glob.glob('images/*.png')
for fname in images[:4]:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (24,17), None)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        cv2.drawChessboardCorners(img, (24,17), corners2, ret)

In [15]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [ ]:
img = cv2.imread('images/img5.png')
h,  w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
x, y, w, h = roi
dst2 = dst[y:y+h, x:x+w]
cv2.namedWindow("Calibrated With 4 images", cv2.WINDOW_NORMAL)
cv2.imshow('Calibrated With 4 images', dst2)
cv2.namedWindow("Calibrated With img1.png", cv2.WINDOW_NORMAL)
cv2.imshow('Calibrated With img1.png', dst1)
cv2.waitKey()
cv2.imwrite('Calibrated With 4 images.png', dst2)
cv2.imwrite('Calibrated With img1.png', dst1)